In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class bsm_options_value:

    def __init__(self, S0, K, T, r, sigma, option_type):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type

    def analytical(self):
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)

        if self.option_type == 'call':
            price = self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)

            delta = norm.cdf(d1)
            gamma = norm.pdf(d1) / (self.S0 * self.sigma * np.sqrt(self.T))
            vega = (self.S0 * norm.pdf(d1) * np.sqrt(self.T)) / 100  # per 1% change in vol
            theta = (1/252) * (-self.S0 * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
                              - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2))  # per business day
            rho = (self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2)) / 100

        elif self.option_type == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)

            delta = norm.cdf(d1) - 1
            gamma = norm.pdf(d1) / (self.S0 * self.sigma * np.sqrt(self.T))
            vega = (self.S0 * norm.pdf(d1) * np.sqrt(self.T)) / 100  # per 1% change in vol
            theta = (1/252) * (-self.S0 * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
                              + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2))  # per business day
            rho = (-self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2)) / 100

        return price , delta, gamma, theta, vega, rho


    def analytical_asian(self):
        #The geometric average of lognormally distributed variables is also lognormal

        # σ_G² = σ² * (2n + 1)/(6n)
        n = self.T * 252
        sigma_G2 = self.sigma**2 * (2 * n + 1) / (6 * n)
        sigma_G = np.sqrt(sigma_G2)

        # G0 = S0 * exp((r - 0.5σ²)(T+1)/(2n) + σ² T (2n+1)/(6n²))
        exponent1 = (self.r - 0.5 * self.sigma**2) * (self.T + 1) / (2 * n)
        exponent2 = self.sigma**2 * self.T * (2 * n + 1) / (6 * n**2)
        G0 = self.S0 * np.exp(exponent1 + exponent2)

        # d1 and d2
        ln_G0_K = np.log(G0 / self.K)
        d1 = (ln_G0_K + 0.5 * sigma_G2 * self.T) / (sigma_G * np.sqrt(self.T))
        d2 = d1 - sigma_G * np.sqrt(self.T)

        if self.option_type == 'call':
            price = np.exp(-self.r * self.T) * (G0 * norm.cdf(d1) - self.K * norm.cdf(d2))
        elif self.option_type == 'put':
            price= np.exp(-self.r * self.T) * (self.K * norm.cdf(-d2) - G0 * norm.cdf(-d1))

        return price #, delta, gamma, theta, vega, rho

    ############################################################################################################

    def char_func_asian(self, u):
        """Characteristic function φ(u) of log(S_T) under risk-neutral BSM model"""
        mu_g = np.log(self.S0) + (self.r - 0.5 * self.sigma**2) * self.T / 2
        sigma_g2 = self.sigma**2 * self.T / 3
        return np.exp(1j * u * mu_g - 0.5 * u**2 * sigma_g2)


    def chi_psi_vectorized(self, k, a, b, c, d):
        omega = k * np.pi / (b - a)      # omega_k
        delta = b - a                    # Precompute width
        # Arguments
        omega_c = omega * (c - a)
        omega_d = omega * (d - a)
        # Avoid division by zero for k=0 in psi
        omega_nonzero = omega.copy()
        omega_nonzero[0] = 1.0
        # ---- chi_k ----
        exp_c = np.exp(c)
        exp_d = np.exp(d)

        chi_k = (exp_d * (np.cos(omega_d) + omega * np.sin(omega_d)) - exp_c * (np.cos(omega_c) + omega * np.sin(omega_c)) )
        chi_k /= (1.0 + omega**2)

        # ---- psi_k ----
        psi_k = (np.sin(omega_d) - np.sin(omega_c)) / omega_nonzero

        # Correct psi_k[0] separately (limit as omega → 0)
        psi_k[0] = d - c

        return chi_k, psi_k

    def fft_cos_asian(self, N=256, L=10):
        # Step 1: log-strike
        x0 = np.log(self.S0)

        # Step 2: Truncation range [a, b]
        c1 = x0 + (self.r - 0.5 * self.sigma**2) * self.T
        c2 = self.sigma**2 * self.T
        c4 = 3 * self.sigma**4 * self.T**2

        a = c1 - L * np.sqrt(abs(c2) + np.sqrt(abs(c4)))
        b = c1 + L * np.sqrt(abs(c2) + np.sqrt(abs(c4)))

        k = np.arange(N)
        u = k * np.pi / (b - a)

        if self.option_type == 'call':
            c, d = np.log(self.K), b
            chi, psi = self.chi_psi_vectorized(k, a, b, c, d)
            Vk = (chi - self.K * psi)
        elif self.option_type == 'put':
            c, d = a, np.log(self.K)
            chi, psi = self.chi_psi_vectorized(k, a, b, c, d)
            Vk = (self.K * psi - chi)
        else:
            raise ValueError("Option type must be 'call' or 'put'.")

        Vk[0] *= 0.5  # k=0 term is halved

        # Step 4: COS formula
        phi = self.char_func_asian(u)
        payoff = np.real( (2  / (b-a)) * phi * np.exp(-1j * u * a)) * Vk
        price = np.exp(-self.r * self.T) * np.sum(payoff)

        return price

    ############################################################################################################
    def plot_option_value_contour(self, S, V, U, S_T=None, V_T=None, option_price_at_ST=None,
                                  ax=None, filled=True, levels=50,
                                  title="Option Price Contour Plot"):

        fig = None
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 7))

        if filled:
            contour = ax.contourf(S, V, U, levels=levels, cmap='viridis')
            fig.colorbar(contour, ax=ax, label='Option Price')
        else:
            contour = ax.contour(S, V, U, levels=levels, cmap='viridis')
            ax.clabel(contour, inline=True, fontsize=8)

        ax.axvline(x=self.K, color='red', linestyle='--', label='Strike Price K')
        if S_T is not None and V_T is not None:
            ax.plot(S_T, V_T, 'ro', label=f'Option Value: {option_price_at_ST:.4f}')
            ax.legend()

        ax.set_xlabel('Asset Price $S$')
        ax.set_ylabel('Variance $v$')
        ax.set_title(title)
        plt.tight_layout()
        plt.show()

    def plot_option_value(self, S, V, S_T, option_price_at_ST, ax):
        ax.plot(S, np.maximum(S - self.K, 0), '--k' , label='Option Value vs Stock Price at maturity')
        ax.plot(S, np.maximum(self.K - S, 0), '--k' , label='Option Value vs Stock Price at maturity')
        ax.plot(S, V, color = 'blue', label='Option Value vs Stock Price at present')
        ax.axvline(x=self.K, color='red', linestyle='--', label='Strike Price K')
        ax.axvline(x=S_T, color='green', linestyle='--', label=f'Stock Price $S_0$')
        ax.scatter([S_T], [option_price_at_ST], color='black', zorder=5, label=f'Option Value at $S_0$')

        ax.set_title('Option Value vs Stock Price at Maturity')
        ax.set_xlabel('Stock Price S')
        ax.set_ylabel('Option Value V')
        ax.set_xlim(self.K - 10 , self.K + 10)
        ax.set_ylim(0, self.K)
        #ax.legend()
        ax.grid(True)

    def make_1D_diff_matrices(self, N, d):
        """Builds centered 1st and 2nd derivative matrices with Dirichlet BCs."""
        e = np.ones(N+1)
        D1 = sp.diags([-e, e], [-1, 1], shape=(N+1, N+1)) / (2 * d)
        D2 = sp.diags([e, -2*e, e], [-1, 0, 1], shape=(N+1, N+1)) / (d ** 2)
        D1 = D1.tolil(); D2 = D2.tolil()
        D1[0, :], D1[-1, :] = 0, 0
        D2[0, :], D2[-1, :] = 0, 0
        return D1.tocsr(), D2.tocsr()


    def psor_cn_2pde(self, A, b, payoff, omega=1.2, tol=1e-2, max_iter=10):
        """
        Projected SOR method to solve (A u >= b, u >= payoff)
        """
        u = payoff.copy()
        N = len(b)

        for it in range(max_iter):
            u_old = u.copy()

            for i in range(N):
                row_start = A.indptr[i]
                row_end = A.indptr[i + 1]
                Ai = A.indices[row_start:row_end]
                Av = A.data[row_start:row_end]

                a_ii = 0.0
                sum_off_diag = 0.0

                for j, a_ij in zip(Ai, Av):
                    if j == i:
                        a_ii = a_ij  # diagonal
                    else:
                        sum_off_diag += a_ij * u[j]  # use current/old u[j] (Gauss-Seidel style)

                if a_ii == 0:
                    continue  # skip if no diagonal

                # Explicitly include (1 - ω)
                u_new = (1 - omega) * u[i] + (omega / a_ii) * (b[i] - sum_off_diag)
                u[i] = max(payoff[i], u_new)

            error = np.linalg.norm(u - u_old, ord=np.inf)
            if error < tol:
                break
#        else:
#            print("⚠️ PSOR did not converge within max_iter")

        return u

    def build_L_asian(self, Nx, Nv, Dx, Dxx, Dv, Dvv, x, v, r, sigma):
        """
        Builds the full 2D operator L using Kronecker products for the Heston PDE.
        """

        # Term 1: 0.5 * sigma**2 * d2/dx2
        L1 = sp.kron(0.5 * sigma**2 * Dxx, sp.eye(Nv+1))

        # Term 2: (r - 0.5sigma**2) * d/dx
        L2 = sp.kron(Dx * (r - 0.5 * sigma**2), sp.eye(Nv+1))

        # Term 3: e**x * d/dv
        exp_x = np.clip(np.exp(x), 1e-6, 10 * self.S0)
        L3 = sp.kron(sp.diags(exp_x) , Dv)

        # Term 6: -r * u
        L4 = -r * sp.eye((Nx+1) * (Nv+1))

        L = L1 + L2 + L3 + L4
        return L.tocsr()

    def crank_nicolson_fd_asian(self, exercise_type, exercise_days):
        # Grid parameters
        Nx, Nv, Nt = 80, 80, 100

        S_max, v_max = 3 * self.S0, 3 * self.S0

        x_min, x_max = np.log(self.S0/10), np.log(S_max)
        v_min = self.S0/10

        dt = self.T / Nt

        x = np.linspace(x_min, x_max, Nx + 1)
        v = np.linspace(v_min, v_max, Nv + 1)
        dx = x[1] - x[0]
        dv = v[1] - v[0]

        X, V = np.meshgrid(x, v, indexing='ij')

        if exercise_type == 'european':
            exercise_binary = np.zeros(Nt + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(Nt + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(Nt + 1)
            for i in range(Nt+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Initial condition (European put)
        S = np.exp(X)

        # Terminal payoff
        if self.option_type == 'call':
            U = np.maximum(V - self.K, 0)
        elif self.option_type == 'put':
            U = np.maximum(self.K - V, 0)

        # Flatten U and prepare identity
        U_flat = U.flatten()
        I = sp.eye((Nx + 1) * (Nv + 1))

        Dx, Dxx = self.make_1D_diff_matrices(Nx, dx)
        Dv, Dvv = self.make_1D_diff_matrices(Nv, dv)

        # Build L once
        L = self.build_L_asian(Nx, Nv, Dx, Dxx, Dv, Dvv, x, v, self.r, self.sigma)

        # Time-stepping loop (Crank-Nicolson)
        A = I - 0.5 * dt * L
        B = I + 0.5 * dt * L

        for n in range(Nt):
            rhs = B @ U_flat

            # Solve A x = rhs
            if exercise_binary[n] == 1:
                payoff = np.maximum(V[:, None] - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - V[:, None], 0)
                payoff_flat = payoff.flatten()
                U_flat = self.psor_cn_2pde(A, rhs, payoff_flat)
            else:
                U_flat = spla.spsolve(A, rhs)

            # Reshape and plot
            U = U_flat.reshape((Nx + 1, Nv + 1))

            # Apply Dirichlet BCs (standard choice: payoff = 0 at boundaries)
            U[0, :] = 0        # x = x_min (S → 0)
            U[-1, :] = U[-1, :] = np.exp(v_max) - self.K if self.option_type == 'call' else 0  # x = x_max (S → ∞)
            U[:, 0] = U[:, 1]  # v = v_min (absorbing or Neumann: du/dv = 0)
            U[:, -1] = U[:, -2]  # v = v_max (same)

            U_flat = U.flatten()

        U = U_flat.reshape((Nx + 1, Nv + 1))

        # Interpolate to get value at (S0, v0)
        interp = RegularGridInterpolator((x, v), U)
        U_price = interp([[np.log(self.S0), self.S0]])[0]

        return U_price, S, V, U, self.S0, self.S0, U_price


    def compute_greeks_fd(self, S, V, method):
        dS = S[1] - S[0]

        # Find closest index to S0
        idx = np.searchsorted(S, self.S0)
        if idx < 1 or idx >= len(S) - 1:
            print("S0 is too close to the boundary for finite difference.")
            return

        # First Derivative: Delta
        delta = (V[idx + 1] - V[idx - 1]) / (2 * dS)

        # Second Derivative: Gamma
        gamma = (V[idx + 1] - 2 * V[idx] + V[idx - 1]) / (dS ** 2)

        # Theta: Approximate using terminal payoff and one time step back
        dt = self.T / 1000
        S_val = S[idx]
        if self.option_type == 'call':
            payoff = max(S_val - self.K, 0)
        else:
            payoff = max(self.K - S_val, 0)
        theta = -(V[idx] - payoff) / dt  # Finite diff in time direction

        # Vega: Bump sigma and reprice using the same method
        eps = 1e-4
        orig_sigma = self.sigma
        self.sigma += eps
        if method == 'explicit_fd':
            V_eps, _, _, _, _ = self.explicit_fd()
        elif method == 'implicit_fd':
            V_eps, _, _, _, _  = self.implicit_fd()
        elif method == 'crank_nicolson_fd':
            V_eps, _, _, _, _  = self.crank_nicolson_fd()
        else:
            raise ValueError("Invalid method for computing Vega")
        self.sigma = orig_sigma
        vega = (V_eps - V[idx]) / eps

        # Rho: Bump interest rate and reprice
        orig_r = self.r
        self.r += eps
        if method == 'explicit_fd':
            V_eps_r, _, _, _, _ = self.explicit_fd()
        elif method == 'implicit_fd':
            V_eps_r, _, _, _, _ = self.implicit_fd()
        elif method == 'crank_nicolson_fd':
            V_eps_r, _, _, _, _ = self.crank_nicolson_fd()

        self.r = orig_r
        rho = (V_eps_r - V[idx]) / eps

        return delta, gamma, theta, vega, rho